In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

#### Pegar as 9 bases de dados, criar uma tabela com as medidas de cada uma das redes, criar um conjunto de treinamento com algumas redes BA,SW,... e depois usando knn classificar elas. Acho que isso da conta.

### Redes para usar

Biologicas --> malaria_genes, plant_pol_robertson, human_brains

https://networks.skewed.de/net/malaria_genes, https://networks.skewed.de/net/plant_pol_robertson, https://networks.skewed.de/net/human_brains

Social --> marvel_partnerships, marvel_universe, football 

https://networks.skewed.de/net/marvel_partnerships, https://networks.skewed.de/net/football, https://networks.skewed.de/net/marvel_universe

Tecnologicas --> power, route_views, jdk

https://networks.skewed.de/net/internet_top_pop, https://networks.skewed.de/net/power, https://networks.skewed.de/net/jdk

In [3]:
#G = nx.read_edgelist("HVR_1.txt", nodetype = int,delimiter =',') #Malaria_genes, ER
#G = nx.read_edgelist("plant_pol_robertson.csv", nodetype = int,delimiter = ',') #plant_pol_robertson
#G = nx.read_edgelist("BNU1_0025864_1_DTI_DS01216.csv", nodetype = int,delimiter = ',',data=(('weight',float),)) #Human Brain
#G = nx.read_edgelist("marvel_partnerships.csv", nodetype = int,delimiter = ',') #Marvel Parternship
G = nx.read_edgelist("marvel_universe.csv", nodetype = int,delimiter = ',') #Marvel Universe
#G = nx.read_edgelist("football.csv", nodetype = int,delimiter = ',') #Football
#G = nx.read_edgelist("power.csv", nodetype = int,delimiter = ',') #Power
#G = nx.read_edgelist("route_views.csv", nodetype = int,delimiter = ',') #Route, classificada com ER
#G = nx.read_edgelist("jdk.csv", nodetype = int,delimiter = ',') #jdk

G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
G = nx.convert_node_labels_to_integers(G, first_label=0)
N = len(G)
M = G.number_of_edges()
print('Number of nodes:', N)
print('Number of edges:', M)

# pos = nx.spring_layout(G)
# nx.draw(G, pos, node_color="b", node_size=10, with_labels=False)

Number of nodes: 19182
Number of edges: 95445


In [4]:
def measures(G): #Incluir centralidade, comunidades e outras medidas.
    def momment_of_degree_distribution(G,m):
        M = 0
        N = len(G)
        for i in G.nodes:
            M = M + G.degree(i)**m
        M = M/N
        return M

    N = len(G)
    M = G.number_of_edges()
    vk = dict(G.degree())
    vk = list(vk.values())
    vk = np.array(vk)
    k1 = momment_of_degree_distribution(G,1)
    k2 = momment_of_degree_distribution(G,2)
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    av_cl = nx.average_clustering(G)
    l = nx.average_shortest_path_length(G)
    r=nx.degree_assortativity_coefficient(G)

    return  k1,k2,variance,av_cl,l,r

In [5]:
k1,k2,variance,av_cl,l,r = measures(G)
X_net = [k1,k2,variance,av_cl,l,r]
print(X_net)

[16.210414452709884, 1140.7130712008502, 877.9355344722247, 0.0, 3.3060426966685705, -0.3152618249680532]


In [6]:
cl = ['ER','WS','BA','CM'] #Colocar outros modelos, como o de configuração, SantoFortunato, entre outros.
#ER networks
X = []
y = []
n_nets = 30
av_degree = k1
p = av_degree/(N-1)
for i in range(0,n_nets):
    GER = nx.gnp_random_graph(N, p, seed=None, directed=False)
    Gcc = sorted(nx.connected_components(GER), key=len, reverse=True)
    GER = GER.subgraph(Gcc[0])
    GER = nx.convert_node_labels_to_integers(GER, first_label=0)
    k1,k2,variance,av_cl,l,r = measures(GER)
    x = [k1,k2,variance,av_cl,l,r]
    X.append(x)
    y.append(0.0)

#WS networks
k = int(av_degree)
p = 0.1 #probability of rewiring
for i in range(0,n_nets):
    GWS = nx.watts_strogatz_graph(N, k, p, seed=None)
    Gcc = sorted(nx.connected_components(GWS), key=len, reverse=True)
    GWS = GWS.subgraph(Gcc[0])
    GWS = nx.convert_node_labels_to_integers(GWS, first_label=0)
    k1,k2,variance,av_cl,l,r = measures(GWS)
    x = [k1,k2,variance,av_cl,l,r]
    X.append(x)
    y.append(1.0)

# BA networks
m = int(av_degree/2)
for i in range(0,n_nets):
    GBA = nx.barabasi_albert_graph(N, m)    
    Gcc = sorted(nx.connected_components(GBA), key=len, reverse=True)
    GBA = GBA.subgraph(Gcc[0])
    GBA = nx.convert_node_labels_to_integers(GBA, first_label=0)
    k1,k2,variance,av_cl,l,r = measures(GWS)
    x = [k1,k2,variance,av_cl,l,r]
    X.append(x)
    y.append(2.0)
    
#Configuration Model
for i in range(0,n_nets):
    

KeyboardInterrupt: 

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

X_net = np.array(X_net)
X_net = X_net.reshape(1,len(X_net)) 
X_net = scaler.transform(X_net)
print('Xnet:', X_net.shape)

k = 5
model = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
model.fit(X,y)
# faz a predição no conjunto de teste

In [ ]:
y_pred = model.predict(X_net) 
print('Classe:', cl[int(y_pred)])